In [1]:
import os
import csv
import pandas as pd
from collections import Counter

In [2]:
ls /media/QNAP/People/BartLiefers/ROP/220118/

ROP/  ROP-ReadingArchive/  readme.txt*


In [3]:
ls /media/QNAP/People/BartLiefers/ROP/220118/ROP/

 Grading/  'Images to label'/   'New images'/                   ext_val_rop/
 Images/   'Independent Test'/  'Study images - do not edit'/


In [4]:
ls /media/QNAP/People/BartLiefers/ROP/220118/ROP/Grading/

CH_ROP_grading.xlsx*  KP_ROP_grading.xlsx*     rop_cleaned_master.csv*
GA/                   SB_ROP_grading.xlsx*     rop_grading_master.xlsx*
JT_ROP_grading.xlsx*  collated_gradings.xlsx*


In [5]:
ls /media/QNAP/People/BartLiefers/ROP/220118/ROP-ReadingArchive/complete/CK/

CK_new_ROP_grading.csv*  Normal/  Plus/  Pre-Plus/  Ungradable/


In [6]:
grading_folder = '/media/QNAP/People/BartLiefers/ROP/220118/ROP/Grading/'
data_folder = '/media/QNAP/People/BartLiefers/ROP/220118/ROP/Images/Media'
images = set([fn for fn in os.listdir(data_folder) if fn.endswith('.png')])

In [7]:
junior_graders = 'CH', 'JT', 'SB' # not KP (replaced with CK)
junior_labels = {}
for g in junior_graders:
    orig_xls = '{}{}_ROP_grading.xlsx'.format(grading_folder, g)
    df = pd.read_excel(orig_xls, engine='openpyxl', index_col=False)
    junior_labels[g] = df
    
    
# replace variants of spelling to make labels consistent
replace = {
    'normal': 'normal',
    'Normal': 'normal',
    'N': 'normal',
    
    
    'ungradableu': 'ungradable',
    'Ungradable': 'ungradable',
    'Ungradeable': 'ungradable',
    'U': 'ungradable',
    'ungradable': 'ungradable',
    
    'plus': 'plus',
    'PP': 'pre-plus',
    'Pre-Plus': 'pre-plus',
    'PrePlus': 'pre-plus',
    'Preplus': 'pre-plus',
    
    'pre-plus': 'pre-plus',
    'P': 'plus',
    'Plus': 'plus',
    'PLUS': 'plus',
}

all_data = {}

for grader in junior_graders:
    data = [(row.Filepath.split('\\')[-1], row.Grade) for i, row in junior_labels[grader].iterrows()]

    for path, g in data:
        # get consistent label
        grading = replace[g] if g in replace else g   
        if grading not in ('normal', 'ungradable', 'plus', 'pre-plus'):
            print('unknown label', path, g)
            continue
        
        # clean path (should match with images on disk)
        path = [p for p in (path, path[2:], (path + '.png'), (path[:-1])) if p in images][0]
        
        # unique key
        entry = path, grader
        if entry in all_data:
            print('found duplicate:')
            print('previous grading, new grading:', all_data[entry], grading)
        
        all_data[entry] = grading

found duplicate:
previous grading, new grading: normal normal
found duplicate:
previous grading, new grading: normal normal
found duplicate:
previous grading, new grading: ungradable ungradable
found duplicate:
previous grading, new grading: normal normal
found duplicate:
previous grading, new grading: normal normal
found duplicate:
previous grading, new grading: normal normal
found duplicate:
previous grading, new grading: pre-plus normal
found duplicate:
previous grading, new grading: ungradable normal
found duplicate:
previous grading, new grading: pre-plus normal


In [8]:
# ADD test set grading
for grader in junior_graders:
    g_folder = '/media/QNAP/People/BartLiefers/ROP/220118/ROP/Independent Test/{}'.format(grader)
    for folder in 'Normal', 'Plus', 'Pre-Plus', 'Ungradable':
        try:
            for file in os.listdir('{}/{}'.format(g_folder, folder)):
                if file in images:
                    all_data[file, grader] = replace[folder]
        except:
            continue

## ADD CK

In [9]:
ck_folders = (
    '/media/QNAP/People/BartLiefers/ROP/220118/ROP-ReadingArchive/complete/CK',
    '/media/QNAP/People/BartLiefers/ROP/220118/ROP-ReadingArchive/CK_test'
)
for ck_folder in ck_folders:
    for folder in 'Normal', 'Plus', 'Pre-Plus', 'Ungradable':
        try:
            for file in os.listdir('{}/{}'.format(ck_folder, folder)):
                if file in images:
                    all_data[file, 'CK'] = replace[folder]
        except:
            continue

In [10]:
Counter([g for (f, g), grading in all_data.items()])

Counter({'CH': 3811, 'JT': 3811, 'SB': 3807, 'CK': 3805})

In [11]:
files = set(f for (f, g), grading in all_data.items())
len(files)

7414

In [12]:
junior_graders = 'CK', 'CH', 'JT', 'SB'
junior_graded = {
    grader: set(f for f in files if (f, grader) in all_data)
    for grader in junior_graders
}
{k: len(v) for k, v in junior_graded.items()}

{'CK': 3805, 'CH': 3811, 'JT': 3811, 'SB': 3807}

In [13]:
junior_agreement = {
    file: {
        grader: all_data[file, grader] for grader in junior_graders
        if (file, grader) in all_data
          }
    for file in files
}

## GA

Original files

```python
ga_orig_file = '/media/QNAP/People/BartLiefers/ROP/220118/ROP/Grading/GA/gagrading1.csv'
with open(ga_orig_file, 'r') as f:
    ga_grading = list(csv.reader(f))

for row in ga_grading[1:]:
    file  = row[0] if row[0] in images else row[0][2:]
    all_data[file, 'GA'] = replace[row[1]]
```

In [14]:
ga_folders = (
    #'/media/QNAP/People/BartLiefers/ROP/220118/ROP/Grading/GA', #ignore?
    '/media/QNAP/People/BartLiefers/ROP/220118/ROP-ReadingArchive/Ms Adams', # CH - JT
    '/media/QNAP/People/BartLiefers/ROP/220118/ROP-ReadingArchive/complete/Ms Adams_cksb_dis' # CK - SB
)
for ga_folder in ga_folders:
    for folder in ('Normal', 'Plus', 'Pre-Plus', 'Pre-plus',
                   'Ungradable', 'Ungradeable', 'normal', 'plus', 'pre-plus', 'ungradable'):
    
        try:
            folder_content = os.listdir('{}/{}'.format(ga_folder, folder))
        except:
            continue
        for file in folder_content:
            if file in images:
                all_data[file, 'GA'] = replace[folder]
        

In [15]:
Counter([all_data[file, 'GA'] for file in files if (file, 'GA') in all_data])

Counter({'normal': 2094, 'plus': 175, 'pre-plus': 234, 'ungradable': 2})

In [22]:
def get_grade(file):
    
    if len(junior_agreement[file].values()) == 1:
        print(file, junior_agreement[file], all_data[file, 'GA'])
        
    junior_labels = set(junior_agreement[file].values())
    if len(junior_labels) == 1:
        # juniors agree
        return list(junior_labels)[0]
    else:
        # juniors disagree
        try:
            return all_data[file, 'GA']
        except:
            print(file, junior_agreement[file])
            return None

In [25]:
group_labels = {file: get_grade(file) for file in files}

4096abc4-d2aa-45e9-b266-9a2e966a74fd.14.png {'SB': 'normal'} normal
115b4818-6213-4bfb-92a7-07f7d7303f53.7.png {'SB': 'normal'} normal


In [27]:
Counter([g for f, g in group_labels.items()])

Counter({'normal': 6220, 'plus': 345, 'ungradable': 487, 'pre-plus': 362})

In [28]:
with open('data_split.csv', 'r') as f:
    reader = csv.reader(f)
    header = next(reader)
    data = list(reader)

In [33]:
header

['file', 'subset', 'CH', 'JT', 'KP', 'SB', 'GA', 'group']

In [34]:
new_data_split = []
for file, subset, CH, JT, KP, SB, GA, group in data:
    row = [file, subset]
    for grader in 'CH', 'JT', 'CK', 'SB', 'GA':
        row.append(all_data[file, grader] if (file, grader) in all_data else '?')
    row.append(group_labels[file])
    new_data_split.append(row)    
new_header = 'file', 'subset', 'CH', 'JT', 'CK', 'SB', 'GA', 'group'

In [35]:
data[0]

['001b3dae-9398-45c3-b271-aae45e7b07df.1.png',
 'fold_0',
 '?',
 '?',
 'normal',
 'normal',
 'normal',
 'normal']

In [36]:
new_data_split[0]

['001b3dae-9398-45c3-b271-aae45e7b07df.1.png',
 'fold_0',
 '?',
 '?',
 'normal',
 'normal',
 '?',
 'normal']

In [39]:
with open('new_data_split.csv', 'w') as f:
    csv.writer(f).writerows([new_header] + new_data_split)

In [38]:
new_header

('file', 'subset', 'CH', 'JT', 'CK', 'SB', 'GA', 'group')

In [60]:
with open('disagreementsGongyu/Gill_regrade_CH-JT_cheat.csv', 'r') as f:
    reader = csv.reader(f)
    header = next(reader)
    data = list(reader)
    
i_CH, i_JT = header.index('CH'), header.index('JT')
len(data), Counter([(row[i_CH], row[i_JT]) for row in data])

(918,
 Counter({('normal', 'ungradable'): 34,
          ('pre-plus', 'plus'): 31,
          ('pre-plus', 'normal'): 66,
          ('ungradable', 'normal'): 667,
          ('normal', 'pre-plus'): 71,
          ('plus', 'pre-plus'): 20,
          ('pre-plus', 'ungradable'): 1,
          ('normal', 'plus'): 16,
          ('plus', 'normal'): 5,
          ('ungradable', 'pre-plus'): 3,
          ('ungradable', 'plus'): 3,
          ('plus', 'ungradable'): 1}))

In [61]:
ch = [junior_data[file, 'CH'] for file in files if (file, 'CH') in junior_data]
jt = [junior_data[file, 'JT'] for file in files if (file, 'JT') in junior_data]
Counter(zip(ch, jt))

Counter({('normal', 'normal'): 2356,
         ('ungradable', 'normal'): 667,
         ('normal', 'pre-plus'): 71,
         ('ungradable', 'ungradable'): 285,
         ('normal', 'plus'): 16,
         ('plus', 'plus'): 101,
         ('pre-plus', 'normal'): 66,
         ('pre-plus', 'pre-plus'): 50,
         ('pre-plus', 'plus'): 31,
         ('normal', 'ungradable'): 34,
         ('plus', 'pre-plus'): 20,
         ('ungradable', 'pre-plus'): 3,
         ('ungradable', 'plus'): 3,
         ('plus', 'normal'): 5,
         ('pre-plus', 'ungradable'): 1,
         ('plus', 'ungradable'): 1})

In [62]:
with open('disagreementsGongyu/Gill_regrade_SB-CK_cheat.csv', 'r') as f:
    reader = csv.reader(f)
    header = next(reader)
    data = list(reader)
    
i_SB, i_CK = header.index('SB'), header.index('CK')
len(data), Counter([(row[i_SB], row[i_CK]) for row in data])

(1456,
 Counter({('normal', 'ungradable'): 178,
          ('ungradable', 'normal'): 73,
          ('pre-plus', 'ungradable'): 3,
          ('pre-plus', 'normal'): 13,
          ('normal', 'pre-plus'): 909,
          ('pre-plus', 'plus'): 177,
          ('normal', 'plus'): 89,
          ('normal', '?'): 2,
          ('ungradable', 'pre-plus'): 3,
          ('plus', 'normal'): 3,
          ('plus', 'pre-plus'): 6}))

In [63]:
sb = [junior_data[file, 'SB'] for file in files if (file, 'SB') in junior_data]
ck = [junior_data[file, 'CK'] for file in files if (file, 'CK') in junior_data]
Counter(zip(sb, ck))

Counter({('normal', 'pre-plus'): 840,
         ('ungradable', 'ungradable'): 36,
         ('normal', 'normal'): 1560,
         ('pre-plus', 'plus'): 37,
         ('normal', 'ungradable'): 295,
         ('normal', 'plus'): 326,
         ('ungradable', 'normal'): 146,
         ('pre-plus', 'pre-plus'): 71,
         ('ungradable', 'pre-plus'): 71,
         ('pre-plus', 'normal'): 155,
         ('pre-plus', 'ungradable'): 25,
         ('ungradable', 'plus'): 23,
         ('plus', 'normal'): 70,
         ('plus', 'pre-plus'): 29,
         ('plus', 'plus'): 13,
         ('plus', 'ungradable'): 13})